We first get necessary external data and code

In [82]:
!git clone https://github.com/AllenInstitute/deepinterpolation.git
!mkdir -p ephys

fatal: destination path 'deepinterpolation' already exists and is not an empty directory.


Install deepinterpolation package

In [83]:
!pip install git+https://github.com/AllenInstitute/deepinterpolation.git

  Cloning https://github.com/AllenInstitute/deepinterpolation.git to /tmp/pip-req-build-5qf0jam7
  Running command git clone -q https://github.com/AllenInstitute/deepinterpolation.git /tmp/pip-req-build-5qf0jam7
  Created wheel for deepinterpolation: filename=deepinterpolation-0.1.0-cp36-none-any.whl size=21726 sha256=ffeedf4fed39bc8d211a523b1cb6244ebaf13ba60b85d8023e4c8e98bd9265fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-uf59wkv4/wheels/f1/78/fd/1d2e8acccfd7c204055c5ab9dbc5add837981811641c01dbf1
Successfully built deepinterpolation


In [1]:
import os
from deepinterpolation.generator_collection import SingleTifGenerator
from deepinterpolation.trainor_collection import core_trainer
from deepinterpolation.network_collection import unet_single_1024
import datetime

This is used for record-keeping


In [2]:
now = datetime.datetime.now()
run_uid = now.strftime("%Y_%m_%d_%H_%M")

Initialize meta-parameters objects

In [3]:
training_param = {}
generator_param = {}
network_param = {}
generator_test_param = {}

An epoch is defined as the number of batches pulled from the dataset. Because our datasets are VERY large. Often, we cannot
go through the entirity of the data so we define an epoch slightly differently than is usual.

In [4]:
steps_per_epoch = 5

Those are parameters used for the Validation test generator. Here the test is done on the beginning of the data but
this can be a separate file.


In [6]:
generator_test_param[
    "pre_post_frame"
] = 30  # Number of frame provided before and after the predicted frame
# In practice the test and validation would point at different files or different portion of files. But for demo, we have little data available
generator_test_param["train_path"] = os.path.join(
    "..",
    "sample_data",
    "ophys_tiny_761605196.tif",
)
generator_test_param["batch_size"] = 5
generator_test_param["start_frame"] = 0
generator_test_param["end_frame"] = 99
generator_test_param[
    "pre_post_omission"
] = 0  # Number of frame omitted before and after the predicted frame
generator_test_param["steps_per_epoch"] = -1  # No step necessary for testing as epochs are not relevant. -1 deactivate it.

Those are parameters used for the main data generator

In [7]:
# In practice the test and validation would point at different files or different portion of files. But for demo, we have little data available
generator_param["steps_per_epoch"] = steps_per_epoch
generator_param["pre_post_frame"] = 30
generator_param["train_path"] = os.path.join(
    "..",
    "sample_data",
    "ophys_tiny_761605196.tif",
)
generator_param["batch_size"] = 5
generator_param["start_frame"] = 0
generator_param["end_frame"] = 99
generator_param["pre_post_omission"] = 1

Those are parameters used for the network topology

Those are parameters used for the training process

In [8]:
training_param["run_uid"] = run_uid
training_param["batch_size"] = generator_test_param["batch_size"]
training_param["steps_per_epoch"] = steps_per_epoch
training_param[
    "period_save"
] = 25  # network model is potentially saved during training between a regular nb epochs
training_param["nb_gpus"] = 0
training_param["apply_learning_decay"] = 0
training_param[
    "nb_times_through_data"
] = 1  # if you want to cycle through the entire data. Two many iterations will cause noise overfitting
training_param["learning_rate"] = 0.0001
training_param["pre_post_frame"] = generator_test_param["pre_post_frame"]
training_param["loss"] = "mean_absolute_error"
training_param[
    "nb_workers"
] = 1  # this is to enable multiple threads for data generator loading. Useful when this is slower than training

training_param["model_string"] = (
    "unet_single_1024_"
    + training_param["loss"]
    + "_"
    + training_param["run_uid"]
)

Where do you store ongoing training progress

In [9]:
jobdir = os.path.join(
    training_param["model_string"] + "_" + run_uid,
)
training_param["output_dir"] = jobdir

try:
    os.mkdir(jobdir)
except:
    print("folder already exists")

Here we create all json files that are fed to the training. This is used for recording purposes as well as input to the training proces

Here we create all objects for training.

In [10]:
# We build the generators object. This will, among other things, calculate normalizing parameters.
train_generator = SingleTifGenerator(generator_param)
test_generator = SingleTifGenerator(generator_test_param)

# We build the training object.
training_class = core_trainer(train_generator, test_generator, unet_single_1024({}), training_param)

Start training. This can take very long time.

In [11]:
training_class.run()

5/5 [==============================] - 151s 31s/step - loss: 904.7095 - val_loss: 0.6580


Finalize and save output of the training.

In [12]:
training_class.finalize()

/Users/jerome.lecoq/opt/miniconda3/envs/deepinterpolation_new/lib/python3.10/site-packages/keras-2.15.0-py3.10.egg/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saved model to disk
